## Login and pip

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

In [3]:
! pip install jalali_pandas

## Import Libraries

In [4]:
import pandas as pd
import numpy as np

In [5]:
from datetime import datetime

In [6]:
import jalali_pandas

In [7]:
from transformers import pipeline

## Read Comments

In [8]:
pd.options.display.max_columns= None
pd.options.display.max_colwidth= None
pd.options.display.max_rows = None

In [9]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Chase-Mobile.xlsx')
app = 'chase'
df.head(5)

,Unnamed: 0,username,image,comment,date,rating,helpful
0,0,Travis Duryea,https://play-lh.googleusercontent.com/a-/ALV-UjXDTipEk-c_xnFps5eRdoPksUhhuQ-qDfQF-Qo3biZEEw=s32-rw,"The app is a bit over cluttered and full ads for other Chase services. But it does get the basic job done I guess. The biggest disappointment in the app though is that it doesn't have a widget to show your recent account transaction history. I shouldn't have to log into the just to see where the money in my account is going, at a glance. I would like to see that information on my Android home screen / desktop. So that I can quickly catch any strange transactions.","December 17, 2023",2,171
1,1,Andrew Berry,https://play-lh.googleusercontent.com/a/ACg8ocImBqvPj6lofurnYCSYr3p-WxLIQflItUCjILZEfmRi=s32-rw-mo,"Been a Chase CC customer since 2013. The app is decent, but I think there should be more stats available, especially about cash back. I could've swore they had a more available breakdown before. My BIGGEST issue is that 99% of the time I go to pay using a custom amount the keyboard won't pop up and then it often freezes. I'm just trying to pay my bill...","December 13, 2023",3,191
2,2,Nick Hooker,https://play-lh.googleusercontent.com/a-/ALV-UjW9AEvTGAiGOCp3-LnQFC-AegFqPWdu-8rRjNuxupwTJk0=s32-rw,"Most of the features on the app are solid, and fairly straightforward to use. Love Chase credit cards too, but I can't give more than two stars for one huge reason: you CANNOT edit/cancel recurring transfers. On the app OR the website. How asinine is that?? Whoever is in charge of app/web development is failing miserably, this is such a basic and necessary feature... Get it together Chase!","December 2, 2023",2,212
3,3,Peggy Fischer,https://play-lh.googleusercontent.com/a-/ALV-UjW11i0-EEkJgOBcjKGHRWjhPkV6Uratj-GeAAaDglTHr3c=s32-rw,"I would like to give no star as a business client. App has bugs when sending secure messages. Can't upload PDFs only photos I need to take while using the app. Otherwise the app would crash. The Chase business website is also not properly working. You can at least upload a PDF document but when I want to write a text message, it doesn't let me. And without a text message your can't send it.","December 11, 2023",1,42
4,4,r dupras,https://play-lh.googleusercontent.com/a/ACg8ocKUEQZiuEn-IimJGZcujjpqLYr0zlB8Cyc7pz7eFo1t=s32-rw-mo,"The credit card information is so variable. I've had a charge show up in pending one day, disappear the next, past the third. It's frustrating trying to figure out the balance in order to pay it in full. I've relied on the amount to have a lost charge show up and end up with a balance. Is simple math. You should be able to see everything and calculate additions and subtractions. Not rocket science.","December 7, 2023",3,253


# Preprocessing

In [10]:
df.columns

Index(['Unnamed: 0', 'username', 'image', 'comment', 'date', 'rating',
       'helpful'],
      dtype='object')

In [11]:
columns = df.columns
desired_columns = ['comment', 'rating', 'date','helpful']

for x in columns:
  if x not in desired_columns:
    df.drop(x,axis=1, inplace= True)

In [12]:
df.columns

Index(['comment', 'date', 'rating', 'helpful'], dtype='object')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7320 entries, 0 to 7319
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  7320 non-null   object
 1   date     7320 non-null   object
 2   rating   7320 non-null   int64 
 3   helpful  7320 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 228.9+ KB


# Convert Georgian to Jalali

In [14]:
df['date'] = df['date'].apply(lambda x: datetime.strptime(x,'%B %d, %Y'))

In [15]:
df["date"] = df["date"].jalali.to_jalali()
## Convert Georgian to Jalali
df["year"] = df["date"].jalali.year
df["month"] = df["date"].jalali.month
df["day"] = df["date"].jalali.day

In [16]:
df.head(5)

,comment,date,rating,helpful,year,month,day
0,"The app is a bit over cluttered and full ads for other Chase services. But it does get the basic job done I guess. The biggest disappointment in the app though is that it doesn't have a widget to show your recent account transaction history. I shouldn't have to log into the just to see where the money in my account is going, at a glance. I would like to see that information on my Android home screen / desktop. So that I can quickly catch any strange transactions.",1402-09-26 00:00:00,2,171,1402,9,26
1,"Been a Chase CC customer since 2013. The app is decent, but I think there should be more stats available, especially about cash back. I could've swore they had a more available breakdown before. My BIGGEST issue is that 99% of the time I go to pay using a custom amount the keyboard won't pop up and then it often freezes. I'm just trying to pay my bill...",1402-09-22 00:00:00,3,191,1402,9,22
2,"Most of the features on the app are solid, and fairly straightforward to use. Love Chase credit cards too, but I can't give more than two stars for one huge reason: you CANNOT edit/cancel recurring transfers. On the app OR the website. How asinine is that?? Whoever is in charge of app/web development is failing miserably, this is such a basic and necessary feature... Get it together Chase!",1402-09-11 00:00:00,2,212,1402,9,11
3,"I would like to give no star as a business client. App has bugs when sending secure messages. Can't upload PDFs only photos I need to take while using the app. Otherwise the app would crash. The Chase business website is also not properly working. You can at least upload a PDF document but when I want to write a text message, it doesn't let me. And without a text message your can't send it.",1402-09-20 00:00:00,1,42,1402,9,20
4,"The credit card information is so variable. I've had a charge show up in pending one day, disappear the next, past the third. It's frustrating trying to figure out the balance in order to pay it in full. I've relied on the amount to have a lost charge show up and end up with a balance. Is simple math. You should be able to see everything and calculate additions and subtractions. Not rocket science.",1402-09-16 00:00:00,3,253,1402,9,16


In [17]:
df_backup = df.copy()

### Limit Time

In [18]:
df.drop(['date'],axis=1,inplace=True)

In [19]:
df = df[df['year']>=1401]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2620 entries, 0 to 7318
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  2620 non-null   object
 1   rating   2620 non-null   int64 
 2   helpful  2620 non-null   int64 
 3   year     2620 non-null   int64 
 4   month    2620 non-null   int64 
 5   day      2620 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 143.3+ KB
